# Setup

In [ ]:
from pathlib import Path

fig_base = Path(".").absolute().parent / "reports" / "figures"

fig_base

# Main results grouped bar chart

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Data
tasks = ["Square table", "Round table", "Lamp", "Stool", "Chair"]
methods = ["BC", "Collect-and-Infer", "Trajectory Augmentation", "Negative guidance"]
# Generating random data for the methods within each task
data = np.random.randint(0, 100, size=(len(tasks), len(methods)))

fig, ax = plt.subplots(figsize=(10, 5))

# Set position of bar on X axis
barWidth = 0.2
r1 = np.arange(len(data))
r2 = [x + barWidth for x in r1]
r3 = [x + barWidth for x in r2]
r4 = [x + barWidth for x in r3]

# Make the plot
ax.bar(
    r1,
    data[:, 0],
    width=barWidth,
    edgecolor="grey",
    label="BC",
)
ax.bar(
    r2,
    data[:, 1],
    width=barWidth,
    edgecolor="grey",
    label="Collect-and-Infer",
)
ax.bar(
    r3,
    data[:, 2],
    width=barWidth,
    edgecolor="grey",
    label="Trajectory Augmentation",
)
ax.bar(
    r4,
    data[:, 3],
    width=barWidth,
    edgecolor="grey",
    label="Negative guidance",
)

# Add xticks on the middle of the group bars
ax.set_xlabel("Tasks", fontweight="bold")
ax.set_xticks([r + barWidth for r in range(len(data))])
ax.set_xticklabels(tasks)
ax.set_ylabel("Success rate (%)")
ax.set_ylim(0, 100)

# Adding the "Placeholder" label
plt.text(
    0.5,
    0.5,
    "Placeholder",
    horizontalalignment="center",
    verticalalignment="center",
    transform=ax.transAxes,
    color="red",
    fontsize=48,
    alpha=0.75,
)


# Create legend & Show graphic
ax.legend()
plt.xticks(rotation=45)
plt.title("Success rate of the methods for each task")
plt.tight_layout()
plt.savefig(fig_base / "success_rate.pdf")
plt.show()

# Compare pretraining vision encoder

In [ ]:
import matplotlib.pyplot as plt

# Data
categories = {
    "RN18, Scratch": 49,
    "RN18, ImageNet": 59,
    "Category 3": 30,
}

# Define the color palette with reduced alpha value
colors = ["#FF6666", "#FF3333", "#FF0000"]

# Create figure and axis
fig, ax = plt.subplots()

# Create bar chart with custom colors
ax.bar(categories.keys(), categories.values(), color=colors, alpha=0.75)

# Set y-axis to display percentages
ax.yaxis.set_major_formatter("{x:.0f}%")

# Add labels and title
ax.set_xlabel("Categories")
ax.set_ylabel("Percentage")
ax.set_title("Bar Chart with Percentages")

# Show the plot
plt.show()

# Analyze the `round_table` augmentation

## Qualitatively analyze trajectories

In [ ]:
from pathlib import Path
from src.visualization.render_mp4 import (
    mp4_from_pickle_jupyter,
    unpickle_data,
    pickle_data,
)
from src.common.files import get_raw_paths
import random
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt
from collections import defaultdict

base_dir = Path("tmp")

In [ ]:
paths = get_raw_paths(
    environment="sim",
    demo_source="augmentation",
    demo_outcome="success",
    task="round_table",
    randomness="low",
)

random.shuffle(paths)

len(paths), paths[:3]

In [ ]:
images = defaultdict(list)

for path in tqdm(paths[:200]):
    data = unpickle_data(path)
    images[data.get("critical_state")].append(data["observations"][0]["color_image2"])

In [ ]:
import matplotlib.pyplot as plt
import cv2

# Iterate over the keys and images in the images dictionary
for i, (key, image_list) in enumerate(images.items()):
    if key is None:
        continue

    # Create a 3-by-3 grid of subplots with no space between axes
    img_list = image_list[:9]

    h, w = img_list[0].shape[:2]

    # Center crop each image to be square
    img_list = [img[:, (w - h) // 2 : (w + h) // 2] for img in img_list]

    # Remove 10 pixels from each edge of each image
    img_list = [img[10:-10, 10:-10] for img in img_list]

    # Concatenate the images into a single 3-by-3 image
    img = [np.concatenate(img_list[i : i + 3], axis=1) for i in range(0, 9, 3)]
    img = np.concatenate(img, axis=0)

    # Create a new figure
    fig, ax = plt.subplots(figsize=(10, 10))

    # Display the image
    ax.imshow(img)

    # Remove the x and y ticks
    ax.set_xticks([])
    ax.set_yticks([])

    # Remove everything
    ax.axis("off")

    # Save without a white edge around the image
    plt.savefig(
        fig_base / f"augmentation_grid_{i}.png",
        bbox_inches="tight",
        pad_inches=0,
    )

    # Show the plot
    plt.show()

## Plot coverage of new trajectories

In [ ]:
import zarr
import matplotlib.pyplot as plt
import numpy as np


from src.common.files import get_processed_paths

In [ ]:
aug_path, teleop_path = sorted(
    get_processed_paths(
        environment="sim",
        demo_source=["teleop", "augmentation"],
        demo_outcome="success",
        task="round_table",
        randomness="low",
    )
)

aug_path, teleop_path

In [ ]:
z_aug = zarr.open(str(aug_path), mode="r")
z_teleop = zarr.open(str(teleop_path), mode="r")

ends_aug = z_aug["episode_ends"][:]
ends_teleop = z_teleop["episode_ends"][:]

pos_teleop = z_teleop["robot_state"][:, :3]
pos_aug = z_aug["robot_state"][:, :3]

# Split the data into episodes
pos_teleop = np.split(pos_teleop, ends_teleop[:-1])
pos_aug = np.split(pos_aug, ends_aug[:-1])

# # Concat them together again
pos_teleop = np.concatenate(pos_teleop)
pos_aug = np.concatenate(pos_aug)

In [ ]:
len(ends_teleop), len(pos_teleop), len(ends_aug), len(pos_aug)

### Plot the state-space coverage in 3D

In [ ]:
import plotly.graph_objects as go

# Create the figure
fig = go.Figure()

# Add teleop scatter plot
fig.add_trace(
    go.Scatter3d(
        x=pos_teleop.T[0],
        y=pos_teleop.T[1],
        z=pos_teleop.T[2],
        mode="markers",
        marker=dict(
            size=2,
            opacity=0.5,
            # color="#BCD3FF",
        ),
        name=f"Teleop (n={len(ends_teleop)})",
    )
)

# Add augmentation scatter plot
fig.add_trace(
    go.Scatter3d(
        x=pos_aug.T[0],
        y=pos_aug.T[1],
        z=pos_aug.T[2],
        mode="markers",
        marker=dict(
            size=2,
            opacity=0.5,
            # color="#FFB8B8",
        ),
        name=f"Augmentation (n={len(ends_aug)})",
    )
)

# Update the layout to make it look nice and square
fig.update_layout(
    scene=dict(
        xaxis_title="x",
        yaxis_title="y",
        zaxis_title="z",
        aspectmode="cube",  # Ensures equal aspect ratio for a square look
        camera=dict(
            up=dict(x=0, y=0, z=0.7),  # Sets the z-axis to be up
            center=dict(x=0, y=0, z=0),  # Centers the view
            eye=dict(x=0.9, y=0.9, z=0.9),  # Adjust these values to zoom in
        ),
    ),
    legend=dict(yanchor="top", y=0.9, xanchor="left", x=0.01, font=dict(size=24)),
    margin=dict(l=0, r=0, b=0, t=0),  # Reduce default margins
    width=800,  # Adjust figure width
    height=800,  # Adjust figure height to make it square
)

fig.update_layout(
    scene=dict(zaxis=dict(range=[0, 0.4]))  # Focuses the z-axis to show only 0 to 50
)


# Show the figure
fig.show()

# To save the figure
fig.write_image(str(fig_base / "teleop_augmentation.pdf"))